# <font color = "red">Classificação de Risco de Crédito</font>

## Uma análise Comparativa Para Prever Inadimplência 

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 

In [34]:
# Imports 
import sys
import os
import pandas as pd
import numpy as np


sys.path.append(os.path.abspath('..'))
from utils import *

In [35]:
df_original = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/risco_credito_original.csv")

In [36]:
df_original.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [37]:
df_original.shape

(32581, 12)

In [38]:
df_original.sample(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
5023,23,39000,RENT,3.0,DEBTCONSOLIDATION,D,5000,15.37,1,0.13,Y,4
25503,28,75000,RENT,0.0,EDUCATION,A,12000,6.03,0,0.16,N,6
2946,26,35000,OWN,NaN,PERSONAL,B,7200,11.11,0,0.21,N,4
20606,33,42000,MORTGAGE,3.0,DEBTCONSOLIDATION,A,13000,7.90,0,0.31,N,6
32320,55,50000,RENT,2.0,HOMEIMPROVEMENT,A,20000,7.51,1,0.40,N,30
2400,23,59000,RENT,7.0,PERSONAL,D,2500,14.96,0,0.04,N,2
23327,32,63000,RENT,9.0,DEBTCONSOLIDATION,B,8000,10.74,1,0.13,N,5
7723,24,54400,MORTGAGE,8.0,DEBTCONSOLIDATION,C,12000,NaN,0,0.22,Y,4
23873,27,27000,RENT,2.0,MEDICAL,B,9750,9.63,1,0.36,N,6
39,23,71500,RENT,3.0,DEBTCONSOLIDATION,D,30000,NaN,1,0.42,N,4


##### Verificando valores únicos, para realizar mapeamento das variaveis

In [39]:
df_original.person_home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

In [40]:
df_original.loan_intent.unique()

array(['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT',
       'DEBTCONSOLIDATION'], dtype=object)

##### Realização de mapeamento no conjunto de dados

In [41]:
# Mudando nome das variáveis
df_mapeado = df_original.rename(columns=map_titulo_variaveis)
df_mapeado.head()

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [42]:
# Traduzindo valor de 2 variaveis categóricas
df_mapeado['Tipo_Moradia'] = df_mapeado['Tipo_Moradia'].map(map_tipo_moradia)
df_mapeado['Motivo_Emprestimo'] = df_mapeado['Motivo_Emprestimo'].map(map_motivo_emprestimo)


In [43]:
df_mapeado.sample(5)

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
9002,23,48000,Aluguel,3.0,Consolidação Dívidas,D,8000,15.58,1,0.17,N,2
20720,30,36000,Aluguel,2.0,Empreendimento,B,5000,10.75,0,0.14,N,5
18805,33,30000,Aluguel,3.0,Médico,A,1200,6.76,0,0.04,N,10
21750,31,49000,Aluguel,5.0,Reformas Domésticas,B,6000,10.59,1,0.12,N,9
14378,22,105000,Hipoteca,6.0,Educação,A,7500,NaN,0,0.07,N,2


In [6]:
df_original.loan_status.value_counts()

loan_status
0    25473
1     7108
Name: count, dtype: int64

In [ ]:
personagem	Idade
pessoa_renda	Renda Anual
pessoa_propriedade_de_casa	Propriedade da casa
pessoa_emp_length	Duração do emprego (em anos)
intenção_de_empréstimo	Intenção de empréstimo
grau_de_empréstimo	Grau de empréstimo
valor_emprestado	Valor do empréstimo
taxa_interna_empréstimo	Taxa de juro
status_do_empréstimo	Status do empréstimo (0 é não inadimplente, 1 é inadimplente)
renda_porcentagem_empréstimo	Renda percentual
cb_pessoa_padrão_em_arquivo	Inadimplência histórica
cb_preson_cred_hist_length	Duração do histórico de crédito